In [3]:
pip install dash

  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/b0/68/781d0026a100106b64e4501c76621dfcd0d3c29a546094fcffaa73037a74/dash-2.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for dash-html-components==2.0.0 from https://files.pythonhosted.org/packages/75/65/1b16b853844ef59b2742a7de74a598f376ac0ab581f0dcc34db294e5c90e/dash_html_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-core-components==2.0.0 from https://files.pythonhosted.org/packages/00/9e/a29f726e84e531a36d56cff187e61d8c96d2cc253c5bcef9a7695acb7e6a/dash_core_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-table==5.0.0 from https://files.pythonhosted.org/packages/da/ce/43f77dc8e7bbad02a9f88d07bf794eaf68359df756a28bb9f2f78e255bb1/dash_table-5.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for retrying from https://files.pythonhosted.org/packages/8f/04/9e36f28be4c0532c0e9207ff9dc01fb

In [22]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Generate sample data for three data sources
np.random.seed(42)
num_rows = 100
data1 = pd.DataFrame({
    'Year': np.random.choice(range(2010, 2021), num_rows),
    'Sales': np.random.randint(10, 100, num_rows),
    'Profit': np.random.uniform(100, 500, num_rows),
    'Expenses': np.random.randint(50, 200, num_rows),
    'Revenue': np.random.uniform(200, 800, num_rows),
    'Orders': np.random.randint(20, 100, num_rows),
    'Customers': np.random.randint(5, 50, num_rows)  # Adding 'Customers' column to data1
})
data2 = pd.DataFrame({
    'Year': np.random.choice(range(2010, 2021), num_rows),
    'Sales': np.random.randint(10, 100, num_rows),
    'Profit': np.random.uniform(100, 500, num_rows),
    'Expenses': np.random.randint(50, 200, num_rows),
    'Revenue': np.random.uniform(200, 800, num_rows),
    'Orders': np.random.randint(20, 100, num_rows),
    'Customers': np.random.randint(5, 50, num_rows)  # Adding 'Customers' column to data2
})
data3 = pd.DataFrame({
    'Year': np.random.choice(range(2010, 2021), num_rows),
    'Sales': np.random.randint(10, 100, num_rows),
    'Profit': np.random.uniform(100, 500, num_rows),
    'Expenses': np.random.randint(50, 200, num_rows),
    'Revenue': np.random.uniform(200, 800, num_rows),
    'Orders': np.random.randint(20, 100, num_rows),
    'Customers': np.random.randint(5, 50, num_rows)  # Adding 'Customers' column to data3
})

# Sample a subset of data for scatter plot
scatter_plot_data = data1.sample(50)

# Initialize Dash app
app = dash.Dash(__name__)

# Define layout of the dashboard
app.layout = html.Div([
    html.H1('Interactive Data Visualization Dashboard', style={'text-align': 'center'}),
    
    # Tabs for selecting data source
    dcc.Tabs(id='data-source-tabs', value='data1', children=[
        dcc.Tab(label='Starbucks', value='data1'),
        dcc.Tab(label='Lassi Corner', value='data2'),
        dcc.Tab(label='Village Rose Milk', value='data3')
    ]),
    
    # Dropdown menu for selecting X-axis attribute
    dcc.Dropdown(
        id='x-axis',
        options=[
            {'label': 'Year', 'value': 'Year'},
            {'label': 'Customers', 'value': 'Customers'}
        ],
        value='Year',  # Default selected value
        className='dropdown'
    ),
    
    # Dropdown menu for selecting Y-axis attribute
    dcc.Dropdown(
        id='y-axis',
        options=[
            {'label': 'Sales', 'value': 'Sales'},
            {'label': 'Profit', 'value': 'Profit'},
            {'label': 'Expenses', 'value': 'Expenses'},
            {'label': 'Revenue', 'value': 'Revenue'},
            {'label': 'Orders', 'value': 'Orders'}
        ],
        value='Sales',  # Default selected value
        className='dropdown'
    ),
    
    # Slider for selecting time period
    dcc.RangeSlider(
        id='time-slider',
        min=min(data1['Year'].min(), data2['Year'].min(), data3['Year'].min()),
        max=max(data1['Year'].max(), data2['Year'].max(), data3['Year'].max()),
        step=1,
        marks={i: f'Year {i}' for i in range(2010, 2021)},
        value=[2010, 2020]
    ),
    
    # Graph components for displaying visualizations
    dcc.Graph(id='scatter-plot', className='graph'),
    dcc.Graph(id='bar-chart', className='graph'),
    dcc.Graph(id='pie-chart', className='graph')
])

# Callback function to update scatter plot based on selected data source, X-axis, Y-axis, and time period
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('data-source-tabs', 'value'),
     Input('x-axis', 'value'),
     Input('y-axis', 'value'),
     Input('time-slider', 'value')]
)
def update_scatter_plot(selected_data, x_axis, y_axis, time_period):
    try:
        # Determine which data source to use
        if selected_data == 'data1':
            data = data1
        elif selected_data == 'data2':
            data = data2
        else:
            data = data3
        
        # Check if the selected Y-axis is available in the data source
        if y_axis not in data.columns:
            raise ValueError(f'Selected Y-axis attribute "{y_axis}" is not available in the data source')
        
        # Filter data based on selected time period
        filtered_data = data.loc[(data['Year'] >= time_period[0]) & (data['Year'] <= time_period[1])]
        
        # Create scatter plot
        fig = go.Figure(
            data=[go.Scatter(x=filtered_data[x_axis], y=filtered_data[y_axis], mode='markers')],
            layout=go.Layout(title='Scatter Plot', xaxis=dict(title=x_axis), yaxis=dict(title=y_axis))
        )
        
        return fig
    except Exception as e:
        print(f'Error updating scatter plot: {str(e)}')
        return go.Figure()  # Return an empty figure or handle the error appropriately

# Callback function to update bar chart based on selected data source, X-axis, Y-axis, and time period
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('data-source-tabs', 'value'),
     Input('x-axis', 'value'),
     Input('y-axis', 'value'),
     Input('time-slider', 'value')]
)
def update_bar_chart(selected_data, x_axis, y_axis, time_period):
    try:
        # Determine which data source to use
        if selected_data == 'data1':
            data = data1
        elif selected_data == 'data2':
            data = data2
        else:
            data = data3
        
        # Check if the selected Y-axis is available in the data source
        if y_axis not in data.columns:
            raise ValueError(f'Selected Y-axis attribute "{y_axis}" is not available in the data source')
        
        # Filter data based on selected time period
        filtered_data = data.loc[(data['Year'] >= time_period[0]) & (data['Year'] <= time_period[1])]

        # Create bar chart
        fig = go.Figure(
            data=[go.Bar(x=filtered_data[x_axis], y=filtered_data[y_axis])],
            layout=go.Layout(title='Bar Chart', xaxis=dict(title=x_axis), yaxis=dict(title=y_axis))
        )
        
        return fig
    except Exception as e:
        print(f'Error updating bar chart: {str(e)}')
        return go.Figure()  # Return an empty figure or handle the error appropriately


# Callback function to update pie chart based on selected data source and time period
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('data-source-tabs', 'value'),
     Input('time-slider', 'value')]
)
def update_pie_chart(selected_data, time_period):
    try:
        # Determine which data source to use
        if selected_data == 'data1':
            data = data1
        elif selected_data == 'data2':
            data = data2
        else:
            data = data3
        
        # Filter data based on selected time period
        filtered_data = data.loc[(data['Year'] >= time_period[0]) & (data['Year'] <= time_period[1])]
        
        # Calculate sum of values for each category
        sum_values = filtered_data.sum()
        
        # Create pie chart
        fig = go.Figure(
            data=[go.Pie(labels=sum_values.index, values=sum_values.values)],
            layout=go.Layout(title='Pie Chart')
        )
        
        return fig
    except Exception as e:
        print(f'Error updating pie chart: {str(e)}')
        return go.Figure()  # Return an empty figure or handle the error appropriately

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
